**Artifical and Computational Intelligence Assignment 1**

Group ID/Number: **ACI_GROUP045**


---

Group Members BITS ID:

**20180AC4538** 

**20180AC4537**

**20180AC4564**

Importing the pandas Library and other required Libraries

In [27]:
import pandas as pd  ######### pandas for data manipulation ,dataframe ,read_csv
import numpy as np  ######### Matrix operation
from math import sin, cos, sqrt, atan2, radians   ######## Maths operation 

# 1) Define the environment representation in the following code block

### Reading the city-data.tsv contains the distances between pairs of cities as per the graph in the documentation

In [28]:
#Defining the goal state
goal = "Chennai"
city_data = pd.read_csv("city-data.tsv", sep="\t") ###### Read City data TSV file to dataframe and print its head
city_data.head()

,city_1,city_2,distance
0,Raichur,Panji,457
1,Raichur,Tirupati,453
2,Raichur,Kurnool,100
3,Panji,Mangalore,365
4,Panji,Bellari,409


### Unique cities identified

In [29]:
unique_col = np.unique(city_data[['city_1','city_2']])  ##### printing unique cities from city1 and city2
print(unique_col)

['Bangalore' 'Bellari' 'Chennai' 'Kozhikode' 'Kurnool' 'Mangalore'
 'Nellore' 'Panji' 'Raichur' 'Tirupati']


### Creating a 10x10 (number of unnique cities = 10) data structure usning dataframe for lookup for the city dist ances. If there doesnt exists a path between any city pair the entry in the matrix is -1

In [30]:
df2 = pd.DataFrame(-1, index=unique_col, columns=unique_col)
def init_path_cost():      ####### function to calculate initial path cost
    for city in unique_col:
        df2[city][city] = 0
    print("-----***-----")
    for index ,row in city_data.iterrows():
        #print(row['city_1'])
        df2[row["city_1"]][row["city_2"]] = row["distance"]
        df2[row["city_2"]][row["city_1"]] = row["distance"]
    print(df2)
init_path_cost()

-----***-----
           Bangalore  Bellari  Chennai  Kozhikode  Kurnool  Mangalore  \
Bangalore          0      311      346        356       -1        352   
Bellari          311        0       -1         -1       -1         -1   
Chennai          346       -1        0         -1       -1         -1   
Kozhikode        356       -1       -1          0       -1        233   
Kurnool           -1       -1       -1         -1        0         -1   
Mangalore        352       -1       -1        233       -1          0   
Nellore           -1       -1      175         -1      325         -1   
Panji             -1      409       -1         -1       -1        365   
Raichur           -1       -1       -1         -1      100         -1   
Tirupati          -1      379      153         -1      340         -1   

           Nellore  Panji  Raichur  Tirupati  
Bangalore       -1     -1       -1        -1  
Bellari         -1    409       -1       379  
Chennai        175     -1       -1       

# 2) Define a function which calculates the heuristic distance from each city to the destination city in the following code block
Calculating the haversine distance between each pair of the cities to the destination

"latlong.tsv" file contains the latitude and longitude of each of the city in the following format: "city name":"lattitude":"longitude"

In [31]:
def init_heuristic():   ####### function to calculate heuristic path cost
    latlong_df = pd.read_csv("latlong.tsv", sep="\t",header=None)
    latlong = {}
    for index, row in latlong_df.iterrows():
        latlong[row[0]] = { "lat":row[1], "lon":row[2]}
    #print(latlong)
    latlong_df = pd.DataFrame(0, index=unique_col, columns=unique_col)
    count = 0;
    for i in latlong:
        for j in latlong:
            if latlong_df[i][j]==0 and i==goal:
                latlong_df[i][j] = haversine(latlong[i]['lat'],latlong[i]['lon'],latlong[j]['lat'],latlong[j]['lon'])
                latlong_df[j][i] = latlong_df[i][j]
                count +=1
    print("-----***-----")
    print(latlong_df)
    print("-----***-----")
    print(count)
    return latlong_df

### Function for calculating distance from each node/city to Destination

In [32]:
####### function to define haversine formula to calculate the great-circle distance between two points 
def haversine(lat1, lon1, lat2, lon2):   
    # approximate radius of earth in km
    R = 6371.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

   # print("Result:", distance)
    return distance
latlong_df = init_heuristic()

-----***-----
           Bangalore  Bellari  Chennai  Kozhikode  Kurnool  Mangalore  \
Bangalore          0        0      290          0        0          0   
Bellari            0        0      427          0        0          0   
Chennai          290      427        0        528      388        586   
Kozhikode          0        0      528          0        0          0   
Kurnool            0        0      388          0        0          0   
Mangalore          0        0      586          0        0          0   
Nellore            0        0      154          0        0          0   
Panji              0        0      744          0        0          0   
Raichur            0        0      468          0        0          0   
Tirupati           0        0      110          0        0          0   

           Nellore  Panji  Raichur  Tirupati  
Bangalore        0      0        0         0  
Bellari          0      0        0         0  
Chennai        154    744      468       

# 3) Implementation of A* Algorithm

In [33]:
def check(node_list, city_node):   
    for node in node_list:
        if node.name == city_node.name:
            return True
    return False

replace function replaces the first node with the second node in the list, this is done for the updation of the optimal path cost

In [34]:
def replace(node_list, city_node1, city_node2): 
    node_list[node_list.index(city_node1)] = city_node2

indexOf function returns the index of the list of node cities for any given city node

In [35]:
def indexOf(node_list, city_node):     
    for node in node_list:
        if node.name == city_node.name:
            return node_list.index(node)

get_children function returns the list of nodes which are adjacent to the given node with the help of df2 dataframe

In [36]:
def get_children(parent):         
    children = []
    # print("Parent:",parent)
    if parent is None:
        return children
    for i in range(0, len(df2[parent].values.tolist())):
        if df2[parent].values.tolist()[i] != -1 and df2.columns[i] != parent:
            children.append(df2.columns[i])
    return children
get_children("Panji")

['Bellari', 'Mangalore', 'Raichur']

### Class Node defining the node for any given city in the search space

In [37]:
class Node:    ####### creating a class 
    def __init__(self, parent, name, fn, path_cost,traversed_path):
        #print("self:",self)
        self.parent = parent
        #print("self.name:",name)
        self.name = name
        self.fn = fn
        self.path_cost = path_cost
        self.children = []
        if traversed_path is None:
            self.traversed_path = [name]
        elif parent is None:
            self.traversed_path = [name]
        else:
            #print("traversed_path::",traversed_path)
            #self.traversed_path = []
            self.traversed_path = traversed_path + [name]
            #print("self.traversed_path::",self.traversed_path)
                

    def init_children(self):
        #print("------\nParent:"+self.name)
        if self.traversed_path is None:
            self.traversed_path = [self]
        # print(get_children(self.name))
        for child in get_children(self.name):
            # print(child+":"+str(df2[self.name][child])+":"+str(latlong_df[child][goal]))
            temp = Node(self.name, child, df2[self.name][child],
                        self.path_cost - latlong_df[self.name][goal] + df2[self.name][child] + latlong_df[child][goal],
                        self.traversed_path)
            self.children.append(temp)
            # print(temp)

    def __repr__(self):
        #temp = [str(elem) for elem in self.traversed_path]
        #print(self)
        return self.name + ":" + str(self.fn) + ":" + str(
            self.path_cost) #+ ":" + ','.join([str(elem) for elem in self.traversed_path])


### The below funciton runs the A* algorithm

### Initialization the search with the start node
prints the start node with the node-name:path-cost:total-cost where<br>node-name := city name <br>path-cost := total cost from the starting node to reach till the node city i.e. g(n) <br> total-cost :=  total cost from the starting node added to the heuristic cost i.e f(n) = g(n) + h(n)

In [38]:
start = Node(None,"Panji",0,latlong_df["Panji"][goal],None)
print(start)
print("traversed_path:",start.traversed_path)
start.init_children()

Panji:0:744
traversed_path: ['Panji']


In [41]:
def A_STAR():       ####### function to implement A* 
    frontier = [start]
    explored = []
    while True:
        print("-------------------------------------------------------------------------")
        if not frontier:
            break
        current = frontier.pop(0)
        print("Current:",current)
        print("Explored:",explored)
        print("Frontier:",frontier)
        if current.name==goal:
            return explored, frontier, current
        explored.append(current)
        #print("Children:",current.children)
        for child in current.children:
            child.init_children()
            #print("child.name:"+child.name+"\tcurrent.name:"+current.name)
            child.fn = child.fn + current.fn
            #print("child.fn:"+str(child.fn)+"\tcurrent.fn:"+str(current.fn))
            #print("child.path_cost:"+str(child.path_cost)+"\tcurrent.path_cost:"+str(current.path_cost))
            if not check(explored,child) and not check(frontier,child):
                #print(child)
                #print("check(frontier,child):",check(frontier,child))
                #print("Temp Frontier:",frontier)
                #print("check(explored,child):",check(explored,child))
                #print("Temp explored:",explored)
                frontier.append(child)
                frontier.sort(key=lambda x: x.path_cost)

            elif check(frontier,child) and frontier[indexOf(frontier,child)].fn>child.fn:
                replace(frontier,frontier[indexOf(frontier,child)],child)
                frontier.sort(key=lambda x: x.path_cost)


### Call your main function/algorithm block in the next code block with appropriate input representation

In [42]:
explored, frontier, current  = A_STAR()

-------------------------------------------------------------------------
Current: Panji:0:744
Explored: []
Frontier: []
-------------------------------------------------------------------------
Current: Bellari:409:836
Explored: [Panji:0:744]
Frontier: [Raichur:457:925, Mangalore:365:951]
-------------------------------------------------------------------------
Current: Tirupati:788:898
Explored: [Panji:0:744, Bellari:409:836]
Frontier: [Raichur:457:925, Mangalore:365:951, Bangalore:720:1010]
-------------------------------------------------------------------------
Current: Raichur:457:925
Explored: [Panji:0:744, Bellari:409:836, Tirupati:788:898]
Frontier: [Chennai:941:941, Mangalore:365:951, Bangalore:720:1010, Nellore:924:1078, Kurnool:1128:1516]
-------------------------------------------------------------------------
Current: Chennai:941:941
Explored: [Panji:0:744, Bellari:409:836, Tirupati:788:898, Raichur:457:925]
Frontier: [Kurnool:557:945, Mangalore:365:951, Bangalore:720:101

In [43]:
print("explored:",explored)
print("frontier:",frontier)
print("current:",current)
print("current:",current.traversed_path)

explored: [Panji:0:744, Bellari:409:836, Tirupati:788:898, Raichur:457:925]
frontier: [Kurnool:557:945, Mangalore:365:951, Bangalore:720:1010, Nellore:924:1078]
current: Chennai:941:941
current: ['Panji', 'Bellari', 'Tirupati', 'Chennai']


## (3.1) Path taken to reach destination from Panaji

In [44]:
current.traversed_path

['Panji', 'Bellari', 'Tirupati', 'Chennai']

## (3.2) Cost of the path

In [45]:
current.path_cost

941

## (3.3) Total Number of nodes vistied to get this state

In [47]:
len(explored)

4